In [1]:
from unityagents import UnityEnvironment
import numpy as np

In [2]:
env = UnityEnvironment(file_name="Tennis_Linux/Tennis.x86_64")
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


In [3]:
from agent import TD3MultiAgent
from multiagent import MultiAgent

In [4]:
import torch
multiagents = MultiAgent(env, TD3MultiAgent)

multiagents.agents[0].actor_target.load_state_dict(torch.load('actor2.pth'))
multiagents.agents[1].actor_target.load_state_dict(torch.load('actor2.pth'))

<All keys matched successfully>

In [5]:
for i in range(1, 15):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(2)                          # initialize the score (for each agent)
    while True:
        
        a1 = multiagents.agents[0].actor_target(torch.FloatTensor(states[0]).to('cuda')).detach().cpu().numpy()
        a2 = multiagents.agents[1].actor_target(torch.FloatTensor(states[1]).to('cuda')).detach().cpu().numpy()
        actions = np.clip(np.concatenate([a1,a2]), -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

KeyboardInterrupt: 